In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from config import config
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback
import stable_baselines3
from config2 import config2

%load_ext tensorboard
import sys
from tqdm.notebook import trange
# from utils import record_videos, show_videos
env_id = "intersection-v0"#"racetrack-v0"
num_cpu = 4
# env = gym.make(env_id, render_mode="rgb_array", config=config
            #    )
# env.unwrapped.configure(config)
env = make_vec_env(env_id, n_envs=num_cpu,env_kwargs={"config":config})

log_dir = "logs"

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment intersection-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [2]:
config

{'observation': {'type': 'GrayscaleObservation',
  'observation_shape': (64, 64),
  'stack_size': 4,
  'weights': [0.2989, 0.587, 0.114],
  'scaling': 1.75},
 'action': {'type': 'ContinuousAction', 'longitudinal': True, 'lateral': True},
 'duration': 13,
 'destination': 'o1',
 'initial_vehicle_count': 150,
 'spawn_probability': 0.7,
 'screen_width': 600,
 'screen_height': 600,
 'centering_position': [0.5, 0.5],
 'scaling': 7.15,
 'collision_reward': -5,
 'high_speed_reward': 0,
 'arrived_reward': 3,
 'normalize_reward': False,
 'simulation_frequency': 15,
 'policy_frequency': 5,
 'offroad_terminal': False}

In [ ]:
%tensorboard --logdir=part3/logs/

In [3]:
ppo_params = {
    "n_steps": 64,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 64,
    'tensorboard_log':log_dir
}

ppo_params2 =  {
  "batch_size": 64,
  "n_steps": 512,
  "gamma": 0.99,
  "gae_lambda": 0.95,
  "n_epochs": 10,
  # "ent_coef": 0.0,
  "ent_coef": 0.01,
  "sde_sample_freq": 4,
  "max_grad_norm": 0.5,
  "vf_coef": 0.5,
  "learning_rate": 0.0001,
  "use_sde": True,
  "clip_range": 0.2,
  'tensorboard_log':log_dir
    
}

ppo_params3 =  {
  "batch_size": 128,
  "n_steps": 512,
  "gamma": 0.8,
  "learning_rate": 0.0005,
  'tensorboard_log':log_dir
    
}

ppo_optuna = {
    "n_steps": 1298,
    "gamma": 0.98,
    "learning_rate": 0.0001291858786170927,
    "ent_coef": 0.0016824274086785702,
    "clip_range": 0.19565974040447376,
    "n_epochs": 1,
    'tensorboard_log':log_dir
    }

ppo_optuna_derniere_chance = {
    "n_steps": 1024,
    "gamma": 0.98,
    "learning_rate": 0.0001291858786170927,
    "ent_coef": 0.0016824274086785702,
    "clip_range": 0.19565974040447376,
    "n_epochs": 3,
    "batch_size": 128,
    'tensorboard_log':log_dir
    }

model = PPO('CnnPolicy',env, verbose=1,**ppo_optuna_derniere_chance)

Using cpu device


c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [12]:
model = PPO.load("test_PPO_intersection_matthias5")

In [4]:
# We create a separate environment for evaluation
eval_env = gym.make(env_id, render_mode="rgb_array", config=config)

# Random Agent, before training
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment intersection-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: -4.0 +/- 2.00


In [5]:
class DoneCallback(BaseCallback):
    def __init__(self, check_freq: int, save_freq: int, save_path: str, verbose=1):
        super(DoneCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_freq = save_freq
        self.save_path = save_path
        self.done_count = 0

    def _on_step(self) -> bool:
        if 'done' in self.locals.keys():
            if self.locals['done']:
                self.done_count += 1

        if self.n_calls % self.check_freq == 0:
            print(f"Step: {self.num_timesteps} Done count: {self.done_count}")

        if self.n_calls % self.save_freq == 0:
            self.model.save(self.save_path + str(self.num_timesteps))

        return True

In [6]:
# callback = DoneCallback(check_freq=128)
callback = DoneCallback(check_freq=128, save_freq=2500, save_path="./models/optuna_last_chance_")
# model.set_env(env)
model.learn(total_timesteps=512*1000,callback=callback, progress_bar=True)
model.save("test_PPO_intersection_matthias5_4")

Logging to logs\PPO_55


Output()

Step: 512 Done count: 5

Step: 1024 Done count: 10

Step: 1536 Done count: 15

...

Step: 138752 Done count: 7762

Step: 139264 Done count: 7810

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.8         |
|    ep_rew_mean          | 2.67        |
| time/                   |             |
|    fps                  | 4           |
|    iterations           | 34          |
|    time_elapsed         | 32638       |
|    total_timesteps      | 139264      |
| train/                  |             |
|    approx_kl            | 0.008442799 |
|    clip_fraction        | 0.102       |
|    clip_range           | 0.196       |
|    entropy_loss         | -2.73       |
|    explained_variance   | 0.612       |
|    learning_rate        | 0.000129    |
|    loss                 | 0.179       |
|    n_updates            | 99          |
|    policy_gradient_loss | 0.00179     |
|    std                  | 0.95        |
|    value_loss           | 0.579       |
-----------------------------------------


Step: 139776 Done count: 7857

Step: 140288 Done count: 7892

In [21]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

Mean reward: 3.1 +/- 0.30


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

while True:
  done = truncated = False
  obs, info = eval_env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = eval_env.step(action)
    eval_env.render()



NameError: name 'eval_env' is not defined

In [ ]:
env = gym.make('highway-fast-v0', render_mode='rgb_array')
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done = env.reset(), False
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()